In [3]:
# Lesson 1: Simple ReAct Agent from Scratch with Enhanced Custom Google Search and Summarization
import openai
import re
import os
import requests
from dotenv import load_dotenv
from googlesearch import search  # Install googlesearch-python package
from urllib.parse import quote
from bs4 import BeautifulSoup  # For HTML parsing

_ = load_dotenv()
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Example chat completion (optional, can be removed if not needed)
chat_completion = client.chat.completions.create(
    model="mistral-nemo",
    messages=[{"role": "user", "content": "Hello world"}]
)
print(chat_completion.choices[0].message.content)


class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            model="mistral-nemo",
            temperature=0,
            messages=self.messages
        )
        return completion.choices[0].message.content


prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

google_search:
e.g. google_search: Python web scraping
Returns the top search results for the query

summarize_articles:
e.g. summarize_articles: https://example.com/article1, https://example.com/article2
Summarizes the content of the provided article URLs.

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dog's weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weighs 51 lbs

You then output:

Answer: A Bulldog weighs 51 lbs
""".strip()


def calculate(expression):
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Error in calculation: {e}"


def average_dog_weight(name):
    weights = {
        "Scottish Terrier": "Scottish Terriers average 20 lbs",
        "Border Collie": "Border Collies average weight is 37 lbs",
        "Toy Poodle": "Toy Poodles average weight is 7 lbs"
    }
    return weights.get(name, "An average dog weighs 50 lbs")


def google_search(query, num_results=5):
    try:
        # Perform the search using googlesearch
        search_results = search(query, tld="com", num=num_results, stop=num_results, pause=2)
        results = []
        for url in search_results:
            results.append(url)
        if not results:
            return "No results found."
        return "\n".join(results)
    except Exception as e:
        return f"Error during search: {e}"


def summarize_articles(urls):
    """
    Summarizes the content of the provided article URLs using OpenAI's API.
    
    Parameters:
        urls (str): A comma-separated string of URLs to summarize.
    
    Returns:
        str: Summarized content of the articles.
    """
    try:
        url_list = [url.strip() for url in urls.split(",")]
        summaries = []

        for url in url_list:
            try:
                response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
                if response.status_code != 200:
                    summaries.append(f"Failed to fetch {url}")
                    continue

                soup = BeautifulSoup(response.text, 'html.parser')

                # Extract text from <p> tags
                paragraphs = soup.find_all('p')
                text = ' '.join([p.get_text() for p in paragraphs])

                # Limit the text to a reasonable length for summarization
                if len(text) > 2000:
                    text = text[:2000] + "..."

                # Summarize using OpenAI
                summary_response = client.chat.completions.create(
                    model="mistral-nemo",
                    messages=[{"role": "user", "content": f"Summarize the following text:\n{text}"}],
                    temperature=0.5,
                    max_tokens=150
                )
                summary = summary_response.choices[0].message.content.strip()
                summaries.append(f"Summary for {url}:\n{summary}")
            except Exception as e:
                summaries.append(f"Error summarizing {url}: {e}")

        if not summaries:
            return "No summaries available."
        return "\n\n".join(summaries)
    except Exception as e:
        return f"Error during summarization: {e}"


known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight,
    "google_search": google_search,
    "summarize_articles": summarize_articles
}

# Regular expression to capture Action and its input
action_re = re.compile(r'^Action: (\w+): (.+)$', re.MULTILINE)


def query(question, max_turns=10):
    """
    Processes the question through the Agent and executes actions as needed.

    Parameters:
        question (str): The question to be answered by the Agent.
        max_turns (int): Maximum number of iterations to prevent infinite loops.
    """
    i = 0
    bot = Agent(prompt)
    next_prompt = f"Question: {question}"
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a)
            for a in result.split('\n')
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action_match = actions[0]
            if action_match:
                action, action_input = action_match.groups()
                if action not in known_actions:
                    raise Exception(f"Unknown action: {action}: {action_input}")
                print(f" -- running {action} {action_input}")
                if action == "google_search":
                    observation = known_actions[action](action_input, num_results=5)
                else:
                    observation = known_actions[action](action_input)
                print("Observation:", observation)
                next_prompt = f"Observation: {observation}"
        else:
            # No more actions, end the loop
            break


# Example usage
if __name__ == "__main__":
    # Ensure you have installed the necessary packages
    # Install via pip:
    # pip install openai googlesearch-python beautifulsoup4 requests python-dotenv

    # Example 1: Dog weight
    # question1 = "How much does a toy poodle weigh?"
    # print("=== Example 1 ===")
    # query(question1)

    # # Example 2: Combined dog weight
    # question2 = "I have 2 dogs, a Border Collie and a Scottish Terrier. What is their combined weight?"
    # print("\n=== Example 2 ===")
    # query(question2)

    # Example 3: Google search with Summarization
    question3 = "What is the latest news in artificial intelligence?"
    print("\n=== Example 3 ===")
    query(question3)


Hello! How are you doing today? I'm here to chat about anything you'd like.

=== Example 3 ===
Thought: I should search for the latest news on artificial intelligence using google_search.
Action: google_search: Latest news in artificial intelligence
PAUSE
 -- running google_search Latest news in artificial intelligence
Observation: https://www.artificialintelligence-news.com/
https://www.artificialintelligence-news.com/artificial-intelligence-news/
https://www.artificialintelligence-news.com/categories/artificial-intelligence/
https://www.artificialintelligence-news.com/about-us/
https://www.artificialintelligence-news.com/news/the-risks-behind-the-generative-ai-craze-why-caution-is-growing/
Thought: I should summarize the top articles from the Google search results to provide a concise update on the latest news in artificial intelligence.
Action: summarize_articles: https://www.artificialintelligence-news.com/news/the-risks-behind-the-generative-ai-craze-why-caution-is-growing/, https